<a href="https://colab.research.google.com/github/rcortezk9/tensorflow_lessons/blob/main/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundameantals in TensorFlow

NLP has the goal of deriving infomration out of natural language (could be seqeuences text or speech).

Another common term for NLP problems is sequence to sequence problems (seq2seq).

## Check for GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-1c7bc50d-7f56-5a61-0eab-edbf70739ef5)


## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2021-05-28 20:49:00--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-05-28 20:49:00 (98.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as diaster or not diaster).

See the original source here: https://www.kaggle.com/c/nlp-getting-started


In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data('nlp_getting_started.zip')

--2021-05-28 20:49:48--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.2.112, 172.217.164.144, 142.250.73.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.2.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2021-05-28 20:49:48 (127 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualize our text samples, we first have to read them in, one way to do so would be to use Python: https://realpython.com/read-write-files-python/

But I prefer to get visual straight away.

So another way to do this is to use pandas...

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv('test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [6]:
# Shuffle Training DataFrame

train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# How many examples of class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [10]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) # create radom indexes not higher
for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f'Target" {target}', '(real diaster)' if target > 0 else '(not real diaster)')
  print(f'text:\n{text}\n')
  print('---\n')

Target" 1 (real diaster)
text:
The @POTUS economy continues to collapse.

---

Target" 0 (not real diaster)
text:
2 great new recipes; mudslide cake and so sorry stew! #GBBO

---

Target" 0 (not real diaster)
text:
Arsonists being blamed for a blaze at a plastics recycling business in Adelaide | @pcaldicott7 reports. #7NewsAdl http://t.co/r1Xwdnvb0g

---

Target" 1 (real diaster)
text:
Read an eyewitness account from #Hiroshima from this day in 1945 http://t.co/QUEDV2xxxX #LIFE

---

Target" 1 (real diaster)
text:
Man killed in crash in Barrington Hills: A Hoffman Estates man was killed in a single-car crash Wednesday afte... http://t.co/b6NphxOrZg

---



 ### Split data into training and validation sets

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_senteces, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                              train_df_shuffled['target'].to_numpy(),
                                              test_size=0.1, # use 10% of training data for valid
                                              random_state=42)

In [13]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_senteces), len(val_labels)

(6851, 6851, 762, 762)

In [14]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenziation - direct mapping of token (a token could be a word or a character) to number
* Embedding - create a matrix of a feature vector for each token ( the size of the feature vector can be defined and this embedding can be learned)



### Text vectorization (tokenization)

The processing of each sample contains the following steps:

1. standardize each sample (usually lowercasing + punctuation stripping)
2. split each sample into substrings (usually words)
3. recombine substrings into tokens (usually ngrams)
4. index tokens (associate a unique int value with each token)
5. transform each sample using this index, either into a vector of ints or a dense float vector.

In [15]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None, # create groups of n-words?
                                    output_mode='int', # how to map tokens to numbers
                                    output_sequence_length=None, # how long do you want your sequences to be
                                    pad_to_max_tokens=True)

In [16]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [17]:
# Setup text vectorization variables
max_vocab_length = 10000 # nax number of words to have in our vacabulary
max_length = 15 # max lenght our sequences will be (e.g. how many words from a Tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [18]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)


In [19]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"

text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [20]:
# Choose a random sentence from the training dataaset and tokenize it
random_sentences = random.choice(train_sentences)
print(f"Original text:\n {random_sentences}\
      \n\nVectorized version:")
text_vectorizer([random_sentences])

Original text:
 And all you girls that smoke.... have you notice that you started to lose ALOT OF WEIGHT ??? HMMM Don't know body want no thin girl tf      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   7,   44,   12, 1288,   16,  271,   24,   12, 2837,   16,   12,
        1109,    5, 1505, 6231]])>

In [21]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in our training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer

To make our embedding, we're going to use TensorFlow's embedding layer: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care most about for our embedding layers:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* `input_length` = length of the sequences being passed to the embedding layer

In [22]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # output shape
                             embeddings_initializer='uniform',
                             input_length=max_length)

embedding

In [23]:
# Get a random sentence from the training set
random_sentences = random.choice(train_sentences)
print(f"Original text:\n {random_sentences}\
        \n\nEmbedded version:")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentences]))
sample_embed

Original text:
 Let's say a tree falls on your fence. Do you know how your homeowners insurance may help? http://t.co/VLaIuvToMM http://t.co/AJpnEBG803        

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03336493,  0.04428312, -0.02679538, ..., -0.02123705,
          0.02065584,  0.02849592],
        [-0.03431238,  0.02092024,  0.01865124, ..., -0.02292901,
         -0.01437484,  0.02073758],
        [-0.04235722,  0.04908985,  0.00989338, ...,  0.00073146,
         -0.04169171,  0.00846232],
        ...,
        [-0.04612358,  0.03805176, -0.02703162, ..., -0.00053651,
         -0.02089868,  0.01791871],
        [ 0.0241647 , -0.04735782,  0.00915328, ..., -0.04136245,
          0.0258777 , -0.01741409],
        [ 0.03432542,  0.03912887,  0.03341052, ..., -0.01698007,
         -0.00970513, -0.04259788]]], dtype=float32)>

In [24]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentences

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.03336493,  0.04428312, -0.02679538,  0.01155373, -0.00413611,
         0.01988536, -0.03089173, -0.01857221,  0.03792355,  0.03004756,
        -0.03713665,  0.041212  ,  0.04491192, -0.03566409, -0.02813673,
         0.02287843, -0.03405845,  0.00039048, -0.01314696,  0.026652  ,
         0.01306493,  0.02659099,  0.04703356, -0.01622101,  0.01747466,
        -0.00363634, -0.01493205, -0.04778249, -0.04095379, -0.04787748,
         0.04879942, -0.03646219, -0.04849381, -0.02084928, -0.00604261,
         0.03884287,  0.04954172, -0.03980169, -0.03446824,  0.01451683,
        -0.03361397, -0.00479833, -0.00189535, -0.03426752,  0.04916235,
        -0.04816943, -0.04278225, -0.0461861 , -0.00690154,  0.03929157,
        -0.03944959,  0.02259297,  0.03550223, -0.00949899,  0.04330892,
        -0.01622518,  0.00265878,  0.0308365 , -0.02192935, -0.04354132,
         0.01374098,  0.02128017,  0.02746907,  0.03071665,  0.03757007,
  

## Modelling a text dataset (running a series of experiments)

Now we've got a way to turn our text sequences into numbers, it's time to start building a serivews of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline), this is form Sklearn ML map: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

How are we going to apporach all of these?

Use the standard steps in modelling with tensorflow:

* Create a model
* Build a model 
* Fit a model
* Evaluate our model



### Model 0: Getting a baseline

As with all machine learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experiments to build upon.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

> 🔑 **Note:** It's common practice to use non-DL algorithms as a baseline because of their speed and then later using DL to see if you can improve upon them.

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ('tfidf', TfidfVectorizer()), # convert words to number using tfidf
                    ('clf', MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [26]:
# Evaluate our baseline model
baseline_score = model_0.score(val_senteces, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [27]:
# Make predictions
baseline_preds = model_0.predict(val_senteces)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

We could evaluate all of our model's predictions with different meics every time, however, this will be cumbersome and could easily be fixed with a function...

Let's create one to compare our model's predictions with the truth labels using the following metrics:
* Accuracy
* Precision
* Recall
* F1-score

> For a deep overview of many different evaluation methods, see the sklearn documentation: https://scikit-learn.org/stable/modules/model_evaluation.html

In [28]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classifiction model.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall, and f1-scrore using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
  model_results = {'accuracy': model_accuracy,
                   'precision': model_precision,
                   'recall': model_recall,
                   'f1': model_f1}
  return model_results                   

In [29]:
# Get baseline results
baseline_results = calculate_results(val_labels, baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 1: A simple dense model

In [30]:
# Create a tensorboard callback (need to crate a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = 'model_logs'

In [31]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1, ), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) #turn the input text into numbers
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to one vector
outputs = layers.Dense(1, activation='sigmoid')(x) # create the output layer, want binary outputs so use sigmoid
model_1 = tf.keras.Model(inputs, outputs, name='model_1_dense')

In [32]:
# Compile model
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [33]:
# Fit the model
model_1_history = model_1.fit(x=train_sentences,
                             y=train_labels,
                             epochs=5,
                             validation_data=(val_senteces, val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR, "model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20210528-210719
Epoch 1/5
215/215 [==============================] - 7s 19ms/step - loss: 0.6120 - accuracy: 0.6919 - val_loss: 0.5372 - val_accuracy: 0.7664
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.4433 - accuracy: 0.8171 - val_loss: 0.4703 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 3s 16ms/step - loss: 0.3480 - accuracy: 0.8594 - val_loss: 0.4580 - val_accuracy: 0.7940
Epoch 4/5
215/215 [==============================] - 3s 16ms/step - loss: 0.2858 - accuracy: 0.8895 - val_loss: 0.4610 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.2388 - accuracy: 0.9108 - val_loss: 0.4745 - val_accuracy: 0.7795


In [34]:
# Check the results
model_1.evaluate(val_senteces, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4745 - accuracy: 0.7795


[0.4745054841041565, 0.7795275449752808]

In [35]:
# Make some predictions and evalaute those
model_1_pred_probs = model_1.predict(val_senteces)
model_1_pred_probs.shape

(762, 1)

In [36]:
# Look at a single prediction
model_1_pred_probs[0]

array([0.42520165], dtype=float32)

In [37]:
# Look at the first 10 predictions
model_1_pred_probs[:10]

array([[0.42520165],
       [0.7782731 ],
       [0.99736196],
       [0.14001863],
       [0.11126409],
       [0.93860555],
       [0.92600405],
       [0.99370736],
       [0.9707783 ],
       [0.30911398]], dtype=float32)

In [38]:
# Convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [39]:
# Calculate our model_1 results
model_1_results = calculate_results(val_labels, model_1_preds)
model_1_results

{'accuracy': 77.95275590551181,
 'f1': 0.7776506830100394,
 'precision': 0.7811151206076258,
 'recall': 0.7795275590551181}

In [40]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [41]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

## Visualizing learnded embeddings


In [42]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])